In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

%matplotlib inline

In [2]:
uniq_id_cust= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_customers_dataset.csv')
orders= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_orders_dataset.csv')
items= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_order_items_dataset.csv')

In [3]:
'''
6. Построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию
R - время от последней покупки пользователя до текущей даты
F - суммарное количество покупок у пользователя за всё время
M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. 
Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.  
'''

'\n6. Построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию\nR - время от последней покупки пользователя до текущей даты\nF - суммарное количество покупок у пользователя за всё время\nM - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. \nДля каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.  \n'

In [4]:
#объединим таблицу с id пользователей и таблицу с заказами
join_item_orders= items.merge(orders, how = 'outer', on ='order_id')

In [5]:
join_item_orders.head ()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [6]:
# приведем  к формуту даты
join_item_orders ['order_approved_at'] = pd.to_datetime(join_item_orders ['order_approved_at'])

In [7]:
# Проверим на отсутствие ID
print('{:,} без наименования Id'.format(join_item_orders[join_item_orders.customer_id.isnull()].shape[0]))

0 без наименования Id


In [8]:
# Посмотрим на временные рамки
print('Заказы с {} по {}'.format(join_item_orders['order_approved_at'].min(),
                                    join_item_orders['order_approved_at'].max()))

Заказы с 2016-09-15 12:16:38 по 2018-09-03 17:40:06


In [9]:
# Уберем из выборки созданные, выставлен счет (неоплачен) недоступные и отмененные 
join_item_orders = join_item_orders\
    .query('order_status != ["unavailable", "created", "invoiced", "canceled"]')\

In [10]:
join_item_orders.head ()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1.0,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1.0,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1.0,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [11]:
# перевели в год, в котором сделан заказ и оплачен
join_item_orders ['year_approved'] = join_item_orders.order_approved_at.dt.year

In [12]:
# Так как обычно для рассчета CRR используется год - рассмотрим целый год из данных - 2017
join_item_orders = join_item_orders.query ('year_approved == "2017.0"')

In [13]:
# Посчитаем частоту покупок клиентами (Frequency)
Frequency = join_item_orders\
    .groupby ('customer_id')\
    .agg ({'order_id': 'count'}).reset_index()
Frequency.head ()

,customer_id,order_id
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,0004164d20a9e969af783496f3408652,1


In [14]:
# Рассчитаем сумму по покупкам для каждого пользователя за всё время (Monetary)
Monetary = join_item_orders\
    .groupby(['customer_id']).agg({'price': lambda x: x.sum()}).reset_index()
Monetary.head()

,customer_id,price
0,00012a2ce6f8dcda20d059ce98491703,89.80
1,000161a058600d5901f007fab4c27140,54.90
2,0001fd6190edaaf884bcaf3d49edf079,179.99
3,0002414f95344307404f0ace7a26f1d5,149.90
4,0004164d20a9e969af783496f3408652,59.99


In [15]:
# Рассчитаем время от последней покупки пользователя до текущей даты (Recency)

In [16]:
# Добавим дату последней покупки + 1. Эта дата будет использоваться в качестве ориентира для расчета показателя недавности.
NOW = join_item_orders ['order_approved_at'].max() + timedelta(days=1)
NOW

Timestamp('2018-01-01 23:32:40')

In [17]:
# Найдем месяц самого последнего заказа по клиентам
Recency = join_item_orders\
    .groupby('customer_id')\
    .agg ({'order_approved_at': 'max'})
Recency.head()

,order_approved_at
customer_id,
00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:35:32
000161a058600d5901f007fab4c27140,2017-07-16 09:55:12
0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:15:20
0002414f95344307404f0ace7a26f1d5,2017-08-17 03:10:27
0004164d20a9e969af783496f3408652,2017-04-12 08:50:12


In [18]:
# Добавим столбец с количеством дней между покупкой и настоящим моментом
Recency['DaysSinceOrder'] = Recency['order_approved_at'].apply(lambda x: (NOW - x).days)

In [19]:
Recency.head ()

,order_approved_at,DaysSinceOrder
customer_id,,
00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:35:32,48
000161a058600d5901f007fab4c27140,2017-07-16 09:55:12,169
0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:15:20,307
0002414f95344307404f0ace7a26f1d5,2017-08-17 03:10:27,137
0004164d20a9e969af783496f3408652,2017-04-12 08:50:12,264


In [20]:
#объединим три получившиеся таблицы
FM = Frequency.merge(Monetary, how = 'outer', on ='customer_id')
RFM = FM.merge(Recency, how = 'outer', on ='customer_id')
RFM.rename(columns={'DaysSinceOrder': 'Недавность', 'order_id': 'Частота', 'price': 'Монетизация'}, inplace=True)
RFM.drop(columns=['order_approved_at'], inplace=True)

In [21]:
RFM.head ()

,customer_id,Частота,Монетизация,Недавность
0,00012a2ce6f8dcda20d059ce98491703,1,89.80,48
1,000161a058600d5901f007fab4c27140,1,54.90,169
2,0001fd6190edaaf884bcaf3d49edf079,1,179.99,307
3,0002414f95344307404f0ace7a26f1d5,1,149.90,137
4,0004164d20a9e969af783496f3408652,1,59.99,264


In [22]:
RFM.groupby ('Частота').agg ({'customer_id': 'count'})

,customer_id
Частота,
1,39726
2,3383
3,600
4,253
5,77
6,65
7,7
8,5
9,2


In [23]:
'''
При проверке мы видим, что кол-во клиентов, которые приобрели товар только 1 раз, превалирует в разы над остальными значениями.
Можем сделать вывод, что данный показатель не будет являться репрезентативным.
Но еще  проверим ниже на квантилях.
'''

'\nПри проверке мы видим, что кол-во клиентов, которые приобрели товар только 1 раз, превалирует в разы над остальными значениями.\nМожем сделать вывод, что данный показатель не будет являться репрезентативным.\nНо еще  проверим ниже на квантилях.\n'

In [24]:
# Используем квантили для разбивки
quintiles = RFM[['Недавность', 'Частота', 'Монетизация']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'Недавность': {0.2: 38.0, 0.4: 94.0, 0.6: 157.0, 0.8: 233.0},
 'Частота': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'Монетизация': {0.2: 39.0, 0.4: 65.0, 0.6: 101.93199999999996, 0.8: 177.99}}

In [25]:
'''
Видим, что у частоты квантили везде 1. Из-за явного неравномерного распределения все сильно смещается к числу покупок = 1.
'''

'\nВидим, что у частоты квантили везде 1. Из-за явного неравномерного распределения все сильно смещается к числу покупок = 1.\n'

In [26]:
'''
Теперь запиишем функцию для присвоения рангов от 1 до 5. 
Чем меньше значение недавности, тем лучше. Чем больше частота и монетизация, тем лучше.
'''

'\nТеперь запиишем функцию для присвоения рангов от 1 до 5. \nЧем меньше значение недавности, тем лучше. Чем больше частота и монетизация, тем лучше.\n'

In [27]:
def r_score(x):
    if x <= quintiles['Недавность'][0.2]:
        return 5
    elif x <= quintiles['Недавность'][0.4]:
        return 4
    elif x <= quintiles['Недавность'][0.6]:
        return 3
    elif x <= quintiles['Недавность'][0.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][0.2]:
        return 1
    elif x <= quintiles[c][0.4]:
        return 2
    elif x <= quintiles[c][0.6]:
        return 3
    elif x <= quintiles[c][0.8]:
        return 4
    else:
        return 5    

In [28]:
# присвоим рейтинги
RFM['R'] = RFM['Недавность'].apply(lambda x: r_score(x))
RFM['F'] = RFM['Частота'].apply(lambda x: fm_score(x, 'Частота'))
RFM['M'] = RFM['Монетизация'].apply(lambda x: fm_score(x, 'Монетизация'))

In [29]:
# И создадим столбец RFM и RM
RFM['RFM Score'] = RFM['R'].map(str) + RFM['F'].map(str) + RFM['M'].map(str)
RFM

,customer_id,Частота,Монетизация,Недавность,R,F,M,RFM Score
0,00012a2ce6f8dcda20d059ce98491703,1,89.80,48,4,1,3,413
1,000161a058600d5901f007fab4c27140,1,54.90,169,2,1,2,212
2,0001fd6190edaaf884bcaf3d49edf079,1,179.99,307,1,1,5,115
3,0002414f95344307404f0ace7a26f1d5,1,149.90,137,3,1,4,314
4,0004164d20a9e969af783496f3408652,1,59.99,264,1,1,2,112
...,...,...,...,...,...,...,...,...
44124,fff4f0c85a4d710abe06599e1ed08a42,1,75.00,282,1,1,3,113
44125,fff675a0d5924b9162b4a1bf410466cd,1,59.90,26,5,1,2,512
44126,fff89c8ed4fcf69a823c1d149e429a0b,1,30.00,16,5,1,1,511
44127,fffa0238b217e18a8adeeda0669923a3,1,35.00,112,3,1,1,311


In [30]:
'''
Если посмотреть детально, то в частоте у нас есть значения 1 и 5. В сегмент 5 попадают все, кто сделал больше 1 покупки.
'''

'\nЕсли посмотреть детально, то в частоте у нас есть значения 1 и 5. В сегмент 5 попадают все, кто сделал больше 1 покупки.\n'

In [31]:
count = RFM. groupby('RFM Score').agg ({'customer_id':'count'}).reset_index()
count.shape

(50, 2)

In [32]:
# Всего получилось 49 сегментов.

In [33]:
'''
Ответ:

Использовали квантили для разделения:
Недавность (R): 1: от 0 до 38 дней, 2: от 39 до 94 дней, 3: от 95 до 157 дней, 4: от 158 до 233 дней, 5: от 234 дней и выше
Частота (F): 1: заказчик сделал одну покупку, 2: -, 3: -, 4: -, 5: заказчик сделал больше одной покупки
Монетизация (M): 1: от 0 до 39, 2: от 40 до 65, 3: от 66 до 101.93, 4: от 101,94 до 178, 5: от 179 и выше}}
                        

Так как сегментов 49 и описывать все имеющиеся - долго и объемно, то приведу для примера 3 из них.

RFM-сегмент 413 (recency=4, frequency=1, monetary=3) имеет границы метрик recency от 158 до 233 дней,
frequency 1 заказ в неделю, monetary от 66 до 101.93 рублей 

RFM-сегмент 351 (recency=3, frequency=5, monetary=1) имеет границы метрик recency от 95 до 157 дней,
frequency больше 1 заказа в неделю, monetary от 0 до 39 рублей. 

RFM-сегмент 115 (recency=1, frequency=1, monetary=5) имеет границы метрик recency от 0 до 38 дней,
frequency 1 заказ в неделю, monetary от 179 и выше рублей. 

'''

'\nОтвет:\n\nИспользовали квантили для разделения:\nНедавность (R): 1: от 0 до 38 дней, 2: от 39 до 94 дней, 3: от 95 до 157 дней, 4: от 158 до 233 дней, 5: от 234 дней и выше\nЧастота (F): 1: заказчик сделал одну покупку, 2: -, 3: -, 4: -, 5: заказчик сделал больше одной покупки\nМонетизация (M): 1: от 0 до 39, 2: от 40 до 65, 3: от 66 до 101.93, 4: от 101,94 до 178, 5: от 179 и выше}}\n                        \n\nТак как сегментов 49 и описывать все имеющиеся - долго и объемно, то приведу для примера 3 из них.\n\nRFM-сегмент 413 (recency=4, frequency=1, monetary=3) имеет границы метрик recency от 158 до 233 дней,\nfrequency 1 заказ в неделю, monetary от 66 до 101.93 рублей \n\nRFM-сегмент 351 (recency=3, frequency=5, monetary=1) имеет границы метрик recency от 95 до 157 дней,\nfrequency больше 1 заказа в неделю, monetary от 0 до 39 рублей. \n\nRFM-сегмент 115 (recency=1, frequency=1, monetary=5) имеет границы метрик recency от 0 до 38 дней,\nfrequency 1 заказ в неделю, monetary от 17